# Sentiment Analysis of Twitter Text

In today’s world, Twitter provides people with a way to publicly express their thoughts on any given subject in a concise, condensed format. This allows us to use tweets as a way to predict users’ thoughts or feelings on a certain subject.

Since the 2016 U.S. election, the influence of social media on society has become more and more concerning. Fake news, hate speech, polarization, and echo chambers attract growing scholarships to pay attention to the discussions in the online space. Understanding the sentimental content on social media is crucial to further analysis

In this project, we are going to compare and contrast two models on the performance of classifying a tweet based on sentiments.

## Load data and pre-processing

In [1]:
# import your libraries here
import pandas as pd
import nltk
import re
import numpy as np
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/zhenguo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df_tweet = pd.read_csv('data/Tweets.csv')

In [45]:
df_tweet

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


## BERTweet
https://huggingface.co/docs/transformers/model_doc/bertweet

In [4]:
from transformers import AutoModel, AutoTokenizer

/opt/anaconda3/envs/class/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-05 10:42:53.536942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Data Cleaning
- remove tweets classified as 'neutral' so that we can perform binary classification
- remove non-string tweets
    - possibly just map these to strings?

In [20]:
# https://stackoverflow.com/questions/39275533/select-row-from-a-dataframe-based-on-the-type-of-the-objecti-e-str
# df[df['A'].apply(lambda x: isinstance(x, str))]
df_tweet_bert = df_tweet[df_tweet['text'].apply(lambda x: isinstance(x, str))].reset_index()
df_tweet_bert = df_tweet_bert[df_tweet_bert['sentiment'] != 'neutral'].reset_index()
#df_tweet_bert = df_tweet.loc[type(df_tweet['text']) == str]

In [21]:
df_tweet_bert

,level_0,index,textID,text,selected_text,sentiment
0,1,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
1,2,2,088c60f138,my boss is bullying me...,bullying me,negative
2,3,3,9642c003ef,what interview! leave me alone,leave me alone,negative
3,4,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
4,6,6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
...,...,...,...,...,...,...
16358,27474,27475,b78ec00df5,enjoy ur night,enjoy,positive
16359,27475,27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
16360,27476,27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
16361,27477,27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive


In [50]:

# from sentence_transformers import SentenceTransformer
roberta_model = SentenceTransformer('paraphrase-distilroberta-base-v1');
def normalize_encode_tweet(tweet):
    norm = tokenizer.normalizeTweet(tweet)
    encoded = roberta_model.encode(norm)
    return encoded

In [32]:
# https://www.geeksforgeeks.org/create-a-new-column-in-pandas-dataframe-based-on-the-existing-columns/
df_tweet_bert['embedding'] =  df_tweet_bert.apply(lambda row: normalize_encode_tweet(row.text), axis=1)

KeyboardInterrupt: 

In [51]:
from tqdm import tqdm

# show progress
tqdm.pandas()

# https://www.geeksforgeeks.org/create-a-new-column-in-pandas-dataframe-based-on-the-existing-columns/
df_tweet_bert['embedding'] =  df_tweet_bert.progress_apply(lambda row: normalize_encode_tweet(row.text), axis=1)

100%|██████████| 16363/16363 [12:04<00:00, 22.57it/s]


In [53]:
# df_tweet_bert.to_csv('tweet_roberta_embeddings.csv', index=False)

In [24]:
df_tweet_bert = pd.read_csv('tweet_roberta_embeddings.csv')

In [25]:
df_tweet_bert.drop(df_tweet_bert.columns[[0, 1, 2]], axis=1,inplace=True)

In [26]:
df_tweet_bert.head()

,textID,text,selected_text,sentiment,embedding
0,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,[ 9.30877551e-02 4.43676770e-01 1.10505581e-...
1,088c60f138,my boss is bullying me...,bullying me,negative,[-2.20891997e-01 -2.87244469e-02 1.46015704e-...
2,9642c003ef,what interview! leave me alone,leave me alone,negative,[ 1.11802444e-02 -4.25624251e-01 1.02491967e-...
3,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,[ 1.77452222e-01 2.84410834e-01 5.99784851e-...
4,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,[-1.04325861e-01 2.68305153e-01 -1.53165251e-...


In [11]:
import statsmodels.formula.api

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [27]:
# test train split
# X = df_tweet_bert['embedding']
# when reading BERT from csv
X = df_tweet_bert['embedding'].apply(lambda s: ([float(x.strip(" \n")) for x in s.strip("[]").split()])).values.tolist()
y = df_tweet_bert['sentiment'].map({'negative': 0, 'positive': 1}).values.tolist()

# train + test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)
# re-split train to have training, validation, testing sets
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state=1) # 0.2/0.8 = 0.25

# train = 60%, val = 20%, test = 20% of original data
# TODO: need higher proportion of training data?

## Train Neural Net

In [28]:
# FOR NEURAL NET - convert to np
X_train_nn = np.asarray(X_train)
y_train_nn = y_train
y_train_nn = np.asarray([np.array(y) for y in y_train_nn])

In [29]:
# FOR NEURAL NET - use validation set in training
X_val_nn = np.asarray(X_val)
y_val_nn = y_val
y_val_nn = np.asarray([np.array(y) for y in y_val_nn])
validation_set = (X_val_nn, y_val_nn)

In [30]:
# FOR NEURAL NET - convert to np
X_test_nn = np.asarray(X_test)
y_test_nn = y_test
y_test_nn = np.asarray([np.array(y) for y in y_test_nn])

In [31]:
# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding

In [32]:
nn = Sequential()
# relu instead of sigmoid increases performance - 2 relu activations leads to overfitting
nn.add(Dense(256, input_shape=(768,), activation='sigmoid', name='input'))
# relu instead of sigmoid increases performance
nn.add(Dense(128, activation='relu', name='hidden'))
nn.add(Dense(1, activation='sigmoid', name='output'))
nn.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

2022-12-05 10:54:34.258308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 256)               196864    
                                                                 
 hidden (Dense)              (None, 128)               32896     
                                                                 
 output (Dense)              (None, 1)                 129       
                                                                 
Total params: 229,889
Trainable params: 229,889
Non-trainable params: 0
_________________________________________________________________


In [34]:
nn.fit(X_train_nn, 
       y_train_nn, 
       epochs = 20, 
       batch_size = 10,
       validation_data = validation_set,
       verbose = 1)

Epoch 1/20
982/982 [==============================] - 2s 1ms/step - loss: 0.5968 - accuracy: 0.7003 - val_loss: 0.5105 - val_accuracy: 0.6783
Epoch 2/20
982/982 [==============================] - 1s 1ms/step - loss: 0.3646 - accuracy: 0.8558 - val_loss: 0.3058 - val_accuracy: 0.8753
Epoch 3/20
982/982 [==============================] - 1s 1ms/step - loss: 0.3048 - accuracy: 0.8721 - val_loss: 0.2848 - val_accuracy: 0.8848
Epoch 4/20
982/982 [==============================] - 1s 1ms/step - loss: 0.2880 - accuracy: 0.8806 - val_loss: 0.2944 - val_accuracy: 0.8808
Epoch 5/20
982/982 [==============================] - 1s 1ms/step - loss: 0.2815 - accuracy: 0.8843 - val_loss: 0.2950 - val_accuracy: 0.8827
Epoch 6/20
982/982 [==============================] - 1s 1ms/step - loss: 0.2748 - accuracy: 0.8841 - val_loss: 0.2738 - val_accuracy: 0.8894
Epoch 7/20
982/982 [==============================] - 1s 1ms/step - loss: 0.2693 - accuracy: 0.8904 - val_loss: 0.2739 - val_accuracy: 0.8891
Epoch 

### Evaluation

In [35]:
# predict x_test values
y_pred_temp = nn.predict(X_test_nn)
# convert from probabilities to 0/1
y_pred_nn = [1 if y > 0.5 else 0 for y in y_pred_temp]

103/103 [==============================] - 0s 731us/step


In [36]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

nn_accuracy = accuracy_score(y_test_nn, y_pred_nn)
nn_precision = precision_score(y_test_nn, y_pred_nn)
nn_recall = recall_score(y_test_nn, y_pred_nn)
nn_f1 = f1_score(y_test_nn, y_pred_nn)

print(f'Neural Net Accuracy:  {nn_accuracy}')
print(f'Neural Net Precision: {nn_precision}')
print(f'Neural Net Recall:    {nn_recall}')
print(f'Neural Net F1 Score:  {nn_f1}')

Neural Net Accuracy:  0.8893981057134128
Neural Net Precision: 0.9359385009609225
Neural Net Recall:    0.8479396401625072
Neural Net F1 Score:  0.8897685749086479


## Train logistic regression model

In [13]:
len(X_train[0]) == len(X_train[2])

True

In [14]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=1)
clf_log_reg = log_reg.fit(X_train, y_train)

/opt/anaconda3/envs/class/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Evaluation

In [15]:
y_pred_log = clf_log_reg.predict(X_test)

In [18]:
log_accuracy = accuracy_score(y_test, y_pred_log)
log_precision = precision_score(y_test, y_pred_log)
log_recall = recall_score(y_test, y_pred_log)
log_f1 = f1_score(y_test, y_pred_log)

print(f'Logistic Regression Accuracy:  {log_accuracy}')
print(f'Logistic Regression Precision: {log_precision}')
print(f'Logistic Regression Recall:    {log_recall}')
print(f'Logistic Regression F1 Score:  {log_f1}')

Logistic Regression Accuracy:  0.8875649251451267
Logistic Regression Precision: 0.9016004742145821
Logistic Regression Recall:    0.8827626233313988
Logistic Regression F1 Score:  0.8920821114369502


# Citation

Download the lexicon from http://www.cs.uic.edu/~liub/FBS/opinion-lexicon-English.rar and extract it into `data/positive-words.txt` and `data/negative-words.txt`.

The following pre-processing steps are inspired from https://towardsdatascience.com/text-normalization-for-natural-language-processing-nlp-70a314bfa646.

We also pre-processed data so that it begins with < s> tokens (and ends with < /s> tokens). Inspired from answer: https://stackoverflow.com/questions/37605710/tokenize-a-paragraph-into-sentence-and-then-into-words-in-nltk

normalize text to regular expression
code from https://gist.github.com/yamanahlawat/4443c6e9e65e74829dbb6b47dd81764a